In [5]:
import os
#os.chdir('C:/Users/Mike/Desktop/word_embeddings-master') # windows dir
os.chdir('/home/mike/Desktop/Word Embeddings') # linux dir
import pandas as pd
from gensim.models import Word2Vec
import random

os.chdir('./Text Processing')
from TextPrep import TextPrep
from get_cosine import cosine_sim
os.chdir('..')

os.chdir('./Meta Data')
from key_words import key_words, key_synonyms, agree_words, agree_synonyms, base_words, base_synonyms
from stop_words import stop_words
os.chdir('..')

# set the number of threads
n_threads = 1

# set the number of permutations to run
permutations = 500


In [6]:
tweet_df = pd.read_csv('Data/replication_text.csv')
# Create lists of text and labels
tweets = tweet_df['text']
labels = tweet_df['party']

# initialize parser for both keywords and base words
keyprep = TextPrep(stopwords = stop_words, key_words = key_words, key_synonyms = key_synonyms)
baseprep = TextPrep(stopwords = stop_words, key_words = base_words, key_synonyms = base_synonyms)
agreeprep = TextPrep(stopwords = stop_words, key_words = agree_words, key_synonyms = agree_synonyms)

# preprocess text
tweets = [keyprep.twitter_preprocess(tweet) for tweet in tweets]

In [7]:
# initialize text parser and pre-process tweets
prep = TextPrep(stopwords = stop_words, key_words = base_words, key_synonyms = base_synonyms)
tweets = [prep.twitter_preprocess(tweet) for tweet in tweets]

In [9]:
%%time
sim = []
for i in range(permutations):
    # permute the labels
    rlabels = random.sample(list(labels), len(labels))
    # Get cosine similarity
    rcosine = cosine_sim(parser = prep, keyword = 'place', text = tweets, labels = rlabels, threads = n_threads)
    # append to list
    sim.append(rcosine)

# save results as a csv
simdf = pd.DataFrame(sim, columns = ['cosine similarity'])
simdf.to_csv('place_permutation.csv', index = False)

CPU times: user 7h 27min 8s, sys: 24min 7s, total: 7h 51min 15s
Wall time: 5h 40min 32s


In [19]:
# save results as a csv
#simdf = pd.DataFrame(sim, columns = ['cosine similarity'])
#simdf.to_csv('Analysis/permutation.csv', index = False)